# Mathematik für Biologiestudierende II

Sommersemester 2024

14.05.2024

&copy; 2024 Prof. Dr. Rüdiger W. Braun 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
sns.set_theme()

# ANOVA

Übersicht

|Verteilungsannahmen               | Test in `scipy.stats` | Post-hoc-Test        | in `scipy.stats` |
|:---------------------------------|:----------------------|:---------------------|:-----------------|
|normalverteilt, homoskedastisch   | `f_oneway`            | unverbundener t-Test | `ttest_ind`      |
|normalverteilt, heteroskedastisch | `alexandergovern`     | Welch-Test           | siehe Lektion 17 |
|nicht normalverteil               | `kruskal`             | Mann-Whitney-Test    | `mannwhitneyu`   |

t-Test, Mann-Whitney-Test, ANOVA und die anderen Tests der Tabelle dienen zum Vergleich von Mittelwerten (im parametrischen Fall) und Medianen (im nicht-parametrischen Fall) von kontinuierlichen Merkmalen

# Tests für kategorielle Daten

# $\chi^2$-Unabhängigkeitsstest

* Der Unabhängigkeitstest überprüft, ob zwei Merkmale stochastisch unabhängig sind
* Die Zufallsvariablen $X_1, \dots, X_n$ sind unabhängig mit gleicher Verteilung
* Die Zufallsvariablen $Y_1, \dots, Y_n$ sind unabhängig mit gleicher, aber möglicherweise anderer Verteilung
* Die Zufallsvariable $X_j$ beschreibt ein Merkmal mit den Ausprägungen $w_1, \dots, w_s$  
* Die Zufallsvariable $Y_j$ beschreibt ein Merkmal mit den Ausprägungen $v_1, \dots, v_r$
* Die Nullhypothese ist $P(X_1=w_\ell,\,Y_1=v_k) = P(X_1=w_\ell) \cdot P(Y_1=v_k)$ für alle Wahlen von $k$ und $\ell$ 

## Beispiel: Geschlechterverteilung in verschiedenen Fächern
   
Die Tafel zeigt die eingeschriebenen Erstsemester in einigen Fächern, sortiert nach Geschlecht

|                | Biologie | Biochemie | Chemie | Pharmazie |
|:---------------|----------|-----------|--------|-----------|
| weiblich       | 235      | 75        | 123    | 50        |
| nicht weiblich | 153      | 32        | 81     | 19        |

<!--| Summe          | 388      | 107       | 204    | 69        |  -->

* Frage: Unterscheiden sich die Geschlechterverhältnisse zwischen den Fächern signifikant zum Niveau $\alpha = 0.05$
* Das ist die Frage nach der Unabhängigkeit der Merkmale "Studienfach" und "Geschlecht"

## Kontingenztafel

* Die oben abgebildete Tafel heißt *Kontingenztafel*
* Wir geben sie als DataFrame ein

In [ ]:
tafel = pd.DataFrame(index=['weiblich', 'nicht weiblich'])
tafel['Biologie'] = [235, 153]
tafel['Biochemie'] = [75, 32]
tafel['Chemie'] = [123, 81]
tafel['Pharmazie'] = [50, 19]
tafel

Die ursprünglcihe Kontingenztafel wird später noch gebraucht

In [ ]:
df = tafel.copy()   #  df = tafel vergibt nur einen zweite Namen

### Spaltensummen

In [ ]:
df.sum()

In [ ]:
df.loc['Summe'] = df.sum()
df

`loc` referenziert eine Zeile

### Zeilensummen

In [ ]:
df.sum(axis=1)

In [ ]:
df['insgesamt'] = df.sum(axis=1)
df

Die Wahrscheinlichkeit, dass eine studierende Person weiblich ist, ist

In [ ]:
p_w = 483 / 768
p_w

Die Wahrscheinlichkeit, dass eine Person Biologie studiert, ist

In [ ]:
p_bio = 388 / 768
p_bio

Wenn Studienwahl und Geschlecht unabhängig sind, würde man im linken oberen Feld der Tafel den Wert

In [ ]:
p_w * p_bio * 768

erwarten

In [ ]:
n = df.loc['Summe'].insgesamt
p_zeile = df.loc['Summe'] / n
p_zeile

In [ ]:
p_spalte = df.insgesamt / n
p_spalte

In [ ]:
erwartet = pd.DataFrame(index = df.index)

In [ ]:
erwartet['Biologie'] = n*p_zeile['Biologie']*p_spalte
erwartet['Biochemie'] = n*p_zeile['Biochemie']*p_spalte
erwartet['Chemie'] = n*p_zeile['Chemie']*p_spalte
erwartet['Pharmazie'] = n*p_zeile['Pharmazie']*p_spalte
erwartet['insgesamt'] = erwartet.sum(axis=1)
erwartet

zum Vergleich

In [ ]:
df - erwartet

Aus diesen Unterschiedenn wird die Statistik berechnet

In [ ]:
tafel2 = (df-erwartet)**2 / erwartet
tafel2

Die Summen müssen neu bestimmt werden

In [ ]:
tafel2.loc['Summe'] = tafel2.sum()
tafel2['insgesamt'] = tafel2.sum(axis=1)
tafel2

Die Summe über alle Einträge ist die Teststatistik

In [ ]:
t = tafel2.loc['Summe'].insgesamt
t

* Die zum $\chi^2$-Unabhängigkeitstest gehörende Verteilung ist die $\chi^2$-Verteilung.
* Sie hat Freiheitsgrade
* Wenn die Kontingenztafel r Zeilen und c Spalten hat, dann ist die Zahl der Freiheitsgrade gleich $(r-1) \cdot (c-1)$

* Im Beispiel ist das $(2-1) \cdot (4-1) = 3$ 

In [ ]:
P = stats.chi2(3)
1 - P.cdf(t)

Nicht signifikant

* Das geht auch automatisch
* Achtung:  Es muss die ursprüngliche Kontingenztafel ohne hinzugefügte Summenzeilen oder -Spalten verwendet werden

In [ ]:
res = stats.chi2_contingency(tafel)
res

In [ ]:
res.pvalue

In [ ]:
pd.DataFrame(res.expected_freq)

## Anwendbarkeit

* Der $\chi^2$-Unabhängigkeitstest beruht auf einer Approximation
* Er ist nur zulässig, wenn alle erwarteten Werte mindestens den Wert 5 haben